In [5]:
import os, shutil
import flopy as fp
import numpy as np
import pyemu
import pandas as pd
import matplotlib.pyplot as plt
import pestools as pt
%matplotlib inline
# Quick check for Windows to make sure input files are Windows type
import platform
if 'window' in platform.platform().lower():
    ppp = 'pest++'
    newln='\n'
else:
    ppp='./pestpp'
    newln='\r\n'
    
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MultipleLocator

base_dir = os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints')
[shutil.copy2(os.path.join(base_dir,cf),cf) for cf in os.listdir(base_dir)];
if not os.path.exists('base_pp_dir'):
    os.mkdir('base_pp_dir')
[shutil.copy2(os.path.join(base_dir,cf),os.path.join('base_pp_dir',cf)) for cf in os.listdir(base_dir)];


setting random seed


In [6]:
inpst = pyemu.Pst('freyberg_pp_reg.pst')

In [9]:
inpst.pestpp_options['n_iter_super'] = 100
inpst.pestpp_options['max_n_super'] = 2

inpst.pestpp_options

{'forecasts': 'rivflux_fore,travel_time,fr03c16,fr04c9',
 'max_n_super': 2,
 'n_iter_base': '-1',
 'n_iter_super': 100,
 'super_eigthres': '1.0e-8'}

In [14]:
inpst.write('freyberg_sing2.pst')

# SINGULAR VALUE DECOMPOSITION 
>## "Singular Value Decomposition. Love it, learn it.
>--Michael Basial

## Linear Algebra is the foundation of much of our maths and modeling. At the basis of this is matrices, which are containing vector information like spatial array of properties, mappings from one set of properties to another, the variability of properties.

## Another example of a matrix is just a photograph. It turns out, much of the information contained in a matrix is redundant. If we think of the columns of a matrix as vectors, they are orthogonal but maybe aren't quite the right basis for the infromation. What if we could find another basis, where we rotate to a more suitable set of orthogonal basis vectors and maybe even stretch them?

## Any matrix can be decomposed into 3 matrices
## <center> $\mathbf{M}=\mathbf{U}\mathbf{S}\mathbf{V}^T$ </center>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os

from PIL import Image

# Let's evaluate how this works by exploring the information content in an image

In [ ]:
photo = Image.open('el_presidente.jpg')
plt.imshow(photo, interpolation='nearest')
plt.axis('off');

## Convert to grayscale

### By converting to grayscale, what we are left with is a matrix of information where each pixel (e.g. a cell in rows/columns of the matrix) has a value between 0 and 255 indicating intensity. This is then just a matrix with information in it.

In [ ]:
photogray = np.array(photo.convert('L'))
plt.imshow(photogray, interpolation='nearest', cmap='gray')
plt.axis('off');

## We can treat this like any matrix and perform SVD

In [ ]:
U, sigma, V = np.linalg.svd(photogray)

In [ ]:
plt.plot(sigma)
plt.grid('on')
plt.title('{0} Singular values in descending order'.format(len(sigma)));

In [ ]:
plt.plot(sigma)
plt.grid('on')
plt.title('{0} Singular values in descending order'.format(len(sigma)));
plt.yscale('log');

## Let's make a little function for using a subset of singular values to reconstitute the image

In [ ]:
def recon_image(U,sigma,V,numsing=1, photo=None, printflag=False):
    reconimg = np.dot(np.dot(U[:,:numsing], np.diag(sigma[:numsing])),V[:numsing,:])

    if printflag==True:
        fig,ax = plt.subplots(ncols=2, figsize=(8,8))
        ax[0].imshow(photo, interpolation='nearest', cmap='gray')
        ax[0].axis('off')
        ax[0].set_title('Original')
        ax[1].imshow(reconimg, interpolation='nearest', cmap='gray')
        ax[1].axis('off')
        ss = 's'
        if numsing==1:
            ss = ''
        ax[1].set_title('Using {0} singular value{1}'.format(numsing,ss))
        plt.tight_layout()
        plt.savefig(os.path.join('pngs','svd_{0}.png'.format(numsing)), bbox_inches='tight', pad_inches=0.2)
        plt.close()
    return reconimg


In [ ]:
rec=recon_image(U,sigma,V,2)
plt.imshow(rec,cmap='gray')
plt.axis('off');

## We can also make images sequentially adding singular values

Note - this requires `ffmpeg` to be installed

Also note the two flags set to `False` for now to be sure that we only remake them if we want to

In [ ]:
import platform
if 'window' in platform.platform().lower():
    pref = ''
else:
    pref = './'
make_images=False
make_movie=False

if make_images:
    if not os.path.exists('pngs'):
        os.mkdir('pngs')
    for i in range(len(sigma)):
        print(i)
        recon_image(U, sigma, V, i+1,photogray, True)

if make_movie:
    if os.path.exists('svdmovie.mp4'):
        os.remove('svdmovie.mp4')
    runstr='{0}ffmpeg -f image2 -r 10  -i pngs/svd_%d.png -vcodec libx264 -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" svdmovie.mp4'.format(pref)
    os.system(runstr)

# Fun times, but what does this have to do with modeling?

In [ ]:
import pyemu

### Let's load up a Jacobian Matrix

In [ ]:
injac = pyemu.Jco.from_binary(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp.jcb'))

In [ ]:
injac.df().head()

In [ ]:
inpst = pyemu.Pst(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp.pst'))

In [ ]:
Q = inpst.observation_data.weight.as_matrix()

In [ ]:
Q = np.diag(Q)
Q.shape

In [ ]:
X = injac.df().as_matrix()

In [ ]:
plt.imshow(np.log(X), interpolation='nearest', cmap='viridis')
plt.colorbar()

## We can form up the normal equations matrix (including weights) and take a look at it

This matrix is $\mathbf{X}^T\mathbf{Q}\mathbf{X}$

In [ ]:
XtQX=X.T.dot(Q).dot(X)

In [ ]:
U, sigma, V = np.linalg.svd(XtQX)

In [ ]:
plt.bar(range(len(sigma)),sigma)
plt.grid('on')

In [ ]:
plt.bar(range(len(sigma)),sigma)
plt.yscale('log')
plt.grid('on')

In [ ]:
plt.imshow(np.log(XtQX), interpolation='nearest', cmap='viridis')
plt.colorbar()

In [ ]:
# select which singular vector you want to plot
SV=2

plt.figure(figsize=(12,4))
plt.bar(list(range(U.shape[0])),U[:,SV-1])
#plt.yscale('log')
plt.xlim([0,72])
plt.xticks(list(range(72)))
plt.title('Singular vector showing parameter contributions to singular vector #{0}'.format(SV))
plt.gca().set_xticklabels(inpst.parameter_data['parnme'], rotation=90);